# Denotation AllenNLP SRL BERT

In [1]:
from allennlp_models.pretrained import load_predictor

/home/elena/anaconda3/lib/python3.9/site-packages/pkg_resources/__init__.py:122: PkgResourcesDeprecationWarning: 4.0.0-unsupported is an invalid version and will not be supported in a future release
  warnings.warn(


In [2]:
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

In [3]:
from checklist.pred_wrapper import PredictorWrapper

In [4]:
import csv

In [5]:
import logging
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').disabled = True 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger('allennlp.common.plugins').disabled = True 
logging.getLogger('allennlp.common.model_card').disabled = True 
logging.getLogger('allennlp.models.archival').disabled = True 
logging.getLogger('allennlp.data.vocabulary').disabled = True 
logging.getLogger('cached_path').disabled = True

In [6]:
srl_predictor = load_predictor('structured-prediction-srl-bert')
output = srl_predictor.predict("John cooks pasta and Anna eats it.")
output

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'verbs': [{'verb': 'cooks',
   'description': '[ARG0: John] [V: cooks] [ARG1: pasta] and Anna eats it .',
   'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'O', 'O', 'O', 'O', 'O']},
  {'verb': 'eats',
   'description': 'John cooks pasta and [ARG0: Anna] [V: eats] [ARG1: it] .',
   'tags': ['O', 'O', 'O', 'O', 'B-ARG0', 'B-V', 'B-ARG1', 'O']}],
 'words': ['John', 'cooks', 'pasta', 'and', 'Anna', 'eats', 'it', '.']}

In [7]:
def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [8]:
d = ["John cooks pasta and Anna eats it."]
pred = predict_and_conf(d)
pred

([{'verbs': [{'verb': 'cooks',
     'description': '[ARG0: John] [V: cooks] [ARG1: pasta] and Anna eats it .',
     'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'O', 'O', 'O', 'O', 'O']},
    {'verb': 'eats',
     'description': 'John cooks pasta and [ARG0: Anna] [V: eats] [ARG1: it] .',
     'tags': ['O', 'O', 'O', 'O', 'B-ARG0', 'B-V', 'B-ARG1', 'O']}],
   'words': ['John', 'cooks', 'pasta', 'and', 'Anna', 'eats', 'it', '.']}],
 array([1.]))

In [9]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][1]['description']

In [10]:
def get_arg(pred, arg_target='ARG1'):
    # we assume one predicate:
    predicate_arguments = pred['verbs'][1]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [11]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][1]['description']
        
    return predicate_structure

In [17]:
def found_arg0_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name'], meta['last_name']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

In [13]:
object = ['it']

In [14]:
def found_arg1_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    object = set([meta['object']])
    arg_0 = get_arg(pred, arg_target='ARG1')

    if arg_0 == object:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg1 = Expect.single(found_arg1_people)

In [18]:
# initialize editor object
editor = Editor()

# create examples
t = editor.template("{first_name} {last_name} cooks pasta and {first_name} {last_name} eats {object}.", object = object, meta=True, remove_duplicates = True, nsamples=1000)

In [16]:
with open('/datasets/denotation_bert.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg1)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/denotation_bert.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp) 

Predicting 998 examples
Test cases:      998
Fails (rate):    0 (0.0%)
['Kathryn', 'Miller', 'cooks', 'pasta', 'and', 'Kathryn', 'Miller', 'eats', 'it', '.'] [ True]
['Samuel', 'Gordon', 'cooks', 'pasta', 'and', 'Samuel', 'Gordon', 'eats', 'it', '.'] [ True]
['Alfred', 'Williams', 'cooks', 'pasta', 'and', 'Alfred', 'Williams', 'eats', 'it', '.'] [ True]
['Frederick', 'Stewart', 'cooks', 'pasta', 'and', 'Frederick', 'Stewart', 'eats', 'it', '.'] [ True]
['Sara', 'Graham', 'cooks', 'pasta', 'and', 'Sara', 'Graham', 'eats', 'it', '.'] [ True]
['Alexander', 'Henderson', 'cooks', 'pasta', 'and', 'Alexander', 'Henderson', 'eats', 'it', '.'] [ True]
['Robin', 'Turner', 'cooks', 'pasta', 'and', 'Robin', 'Turner', 'eats', 'it', '.'] [ True]
['Sophie', 'Henderson', 'cooks', 'pasta', 'and', 'Sophie', 'Henderson', 'eats', 'it', '.'] [ True]
['Sophie', 'Green', 'cooks', 'pasta', 'and', 'Sophie', 'Green', 'eats', 'it', '.'] [ True]
['Frank', 'Marshall', 'cooks', 'pasta', 'and', 'Frank', 'Marshall', 

# End of Notebook